<a href="https://colab.research.google.com/github/huili8905/cs5242_project/blob/Hui-Li/CS5242_Project_1_Version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---


# **Set-up for Google Drive**

---

1.   Load Libraries
2.   Check GPU Setting
3.   Mount Google Drive
4.   Load Dataset

In [0]:
#################################################
#Crash the original RAM to increase to 25GB - if required
#################################################
a = []
while(1):
    a.append("1")

In [0]:
######################
1. #Maintain libraries
######################
#utils
from google.colab import drive,files
import tarfile
from tqdm import tqdm

#wrangling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import os

#pytorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import TensorDataset, DataLoader, Dataset

#keras library
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense,TimeDistributed,LSTM, Dropout, BatchNormalization,AveragePooling1D, MaxPooling1D,GlobalAveragePooling1D, GlobalMaxPooling1D,Input , Conv1D
from keras.layers.merge import concatenate
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras import optimizers

In [0]:
tf.__version__

'1.14.0'

In [0]:
pip install tensorflow==1.14.0

In [0]:
######################
2. #Check GPU Setting
######################
torch. device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [0]:
######################
3. #Mount Google Drive
######################
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
######################
4. #Load Dataset
######################
#Check default location
os.getcwd()

'/content'

In [0]:
#Update your path to your dataset
os.chdir('/gdrive/My Drive/')

In [0]:
#Read read_datasetBreakfast.py from the folder location

from read_datasetBreakfast import load_data, read_mapping_dict , get_label_length_seq
COMP_PATH = '/gdrive/My Drive/'
''' 
training to load train set
test to load test set
'''
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 
actions_dict = read_mapping_dict(mapping_loc)
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos


for split in ['training','test']:
  if  split == 'training':
    train_split =  os.path.join(COMP_PATH, 'splits/train.split1.bundle') #Train Split
    data_feat_train, data_labels, data_labels_loc = load_data(train_split, actions_dict, GT_folder, DATA_folder, datatype = split) #Get features and labels
  if  split == 'test':
    test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
    data_feat_test = load_data(test_split, actions_dict, GT_folder, DATA_folder, datatype = split)

Finish Load the Training data and labels!!!
Finish Load the Test data!!!


---

# **Data Preparation and Exploration**

---

1.   Check Data Shape
2.   How Much Data
3.   No. of Classes
4.   Class Distribution (BY VIDEO)
5.   Class Distribution (BY FRAME)
6.   Get Tensor Data and Labels Ready

In [0]:
######################
1. #Check Data Shape
######################

print("data shape: {} and date type: {}".format(data_feat_train[0].shape,data_feat_train[0].dtype))
print("data shape: {} and date type: {}".format(data_feat_train[1].shape,data_feat_train[0].dtype))

In [0]:
######################
2. #How Much Data
######################
print("Number of Training Data: {}".format(len(data_feat_train)))
print("Number of Training Data Labels: {}".format(len(data_labels)))
print("Number of Test Data: {}".format(len(data_feat_test)))
print("% of Test Data: {}".format(len(data_feat_test)/(len(data_feat_train)+len(data_feat_test))*100))


In [0]:
##Unique Labels in each video
print("Unique Labels in video 0: {} ".format(data_labels[0]))
print("Frames in video 0: {} ".format(len(data_feat_train[0])))

In [0]:
######################
3. #No. of class
###################### 
len(actions_dict)

In [0]:
######################
4. #Class Distribution (BY VIDEO)
######################
def wrangle_class_distribution(data_label_var , actions_dict):
  #check for labels class balance
  data_label_var = pd.DataFrame(data_label_var)
  data_label_var = pd.DataFrame(data_label_var.unstack(level=0)).reset_index()
  data_label_var.drop('level_0',inplace=True,axis=1)
  data_label_var.dropna(inplace=True)
  data_label_var.columns =['sample_no','cat']
  data_label_var['cat'].astype(int)
  
  #mapping for actions (change keys and values)
  actions_dict2 = {y:x for x,y in actions_dict.items()}
  data_label_var['cat_name'] = data_label_var['cat'].map(actions_dict2)

  return data_label_var

In [0]:
%matplotlib inline
sn.set(rc={'figure.figsize':(15.7,8.27)})
Train_Class_Dis = wrangle_class_distribution(data_labels , actions_dict)

sn.countplot(y= 'cat_name',orient= 'h',data=Train_Class_Dis,order = Train_Class_Dis['cat_name'].value_counts().index)
plt.show()

#note: no. of unique labels per label in each video

In [0]:
######################
5. #Class Distribution (BY FRAME)
######################
def get_frame_breakdown(COMP_PATH):
  gt_frame_overall = pd.DataFrame()
  for filesname in os.listdir(COMP_PATH+'/groundTruth'):
    gt_file_level = pd.read_csv(COMP_PATH+'/groundTruth/'+filesname,header=None)
    gt_file_level['file_name'] = filesname
    gt_file_level.columns=['cat_name','file_name']
    
    gt_file_level['cat_no'] = gt_file_level['cat_name'].map(actions_dict)
    gt_frame_overall = pd.concat([gt_frame_overall,gt_file_level], sort=False) 

  return gt_frame_overall

In [0]:
COMP_PATH = '/gdrive/My Drive/'
gt_frame_overall = get_frame_breakdown(COMP_PATH)

%matplotlib inline
sn.set(rc={'figure.figsize':(15.7,8.27)})

sn.countplot(y= 'cat_name',orient= 'h',data=gt_frame_overall,order = gt_frame_overall['cat_name'].value_counts().index)
plt.show()

In [0]:
######################
6. #Get Tensor Data and Labels Ready
######################
#get the labels of each frame
def get_gt(train_split):
   file_ptr = open(train_split, 'r')
   content_all = file_ptr.read().split('\n')[1:-1]
   content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
   all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']
   
   labelframe_breakfast = []
   labelseq_breakfast = []
   labellen_breakfast = []

   for content in content_all:
     file_ptr = open( GT_folder + content, 'r')
     curr_gt = file_ptr.read().split('\n')[:-1]
     curr_gt = list(map(actions_dict.get, curr_gt))

     labelframe_breakfast.append(curr_gt)     
     label_seq, length_seq = get_label_length_seq(curr_gt)
     labelseq_breakfast.append(label_seq)  
     #labellen_breakfast.append(length_seq)  

   return labelframe_breakfast,labelseq_breakfast

labelframe_breakfast ,labelseq_breakfast= get_gt(train_split)

print("data label len: {} ".format(len(labelframe_breakfast)))
print("Video 0 data label len: {} ".format(len(labelframe_breakfast[0])))

data label len: 1460 
Video 0 data label len: 544 


---

# **LSTM / GRU Model**


---

Stratified Sampling /Bootstrapping of frames in each video (if there are too many/little frames) 

In [0]:
segments = []
segment_labels = []
segment_len = []
N = 800

for i, video in enumerate(data_feat_train):
  start_idx = min(data_labels_loc[i])
  end_idx = max(data_labels_loc[i])
  #just to see the length of each video - not required
  segment_len_a = (end_idx - start_idx)
  segment_len.append(segment_len_a)

  if segment_len_a>=N:
    #get frames required
    #get index for stratified index 
    stratified_index_df = pd.DataFrame(labelframe_breakfast[i]).reset_index()
    stratified_index_df = stratified_index_df[stratified_index_df[0]!=0]
    stratified_index = stratified_index_df.groupby(0, group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(stratified_index_df))))).sample(n=N,replace=True).reset_index(drop=True)
    stratified_index.sort_values('index',inplace=True)
    index_list = list(stratified_index['index'])
    
    frame = data_feat_train[i][index_list]
    frame = np.array(frame)
    get_label_list = np.array(labelframe_breakfast[i])
    label= get_label_list[index_list]

    segments.append(frame)
    segment_labels.append(label)

  elif segment_len_a < N:
    stratified_index_df = pd.DataFrame(labelframe_breakfast[i]).reset_index()
    stratified_index_df = stratified_index_df[stratified_index_df[0]!=0]
    stratified_index = stratified_index_df.groupby(0, group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(stratified_index_df))),replace=True)).sample(n=N,replace=True).reset_index(drop=True)
    stratified_index.sort_values('index',inplace=True)
    index_list = list(stratified_index['index'])
    
    frame = data_feat_train[i][index_list]
    frame = np.array(frame)
    get_label_list = np.array(labelframe_breakfast[i])
    label= get_label_list[index_list]
    
    segments.append(frame)
    segment_labels.append(label)

segments = np.array(segments)
print(segments.shape)
segment_labels = np.array(segment_labels)
segment_labels = segment_labels.reshape(1460,N,1)
print(segment_labels.shape)

(1460, 800, 400)
(1460, 800, 1)


In [0]:
segments_len  = len(segments)
n_timesteps = segments[segments_len-1].shape[0]

# define LSTM configuration - hyperparameters

n_neurons = 400
n_neurons2 = 1000

n_batch_size = 25
n_epoch = 100

# create LSTM
model = Sequential()
model.add(Bidirectional(LSTM(n_neurons, input_shape=(n_timesteps, 400), return_sequences=True)))
model.add(BatchNormalization())
model.add(TimeDistributed(Dense(400, activation="relu")))
model.add(TimeDistributed(Dense(48, activation="softmax")))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['sparse_categorical_accuracy'])
# train LSTM
es = EarlyStopping(patience=2)
checkpointer = ModelCheckpoint(filepath='/model2.hdf5', verbose=1, save_best_only=True)

model.fit(segments, segment_labels, epochs=n_epoch, batch_size=n_batch_size, callbacks=[es,checkpointer], validation_split=0.2)
print(model.summary())
model.reset_states()


ERROR! Session/line number was not unique in database. History logging moved to new session 60





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 1168 samples, validate on 292 samples
Epoch 1/100
1168/1168 [==============================] - 329s 282ms/step - loss: 3.5716 - sparse_categorical_accuracy: 0.1366 - val_loss: 3.0681 - val_sparse_categorical_accuracy: 0.2242

Epoch 00001: val_loss improved from inf to 3.06808, saving model to /model2.hdf5
Epoch 2/100
1168/1168 [==============================] - 328s 281ms/step - loss: 2.8448 - sparse_categorical_accuracy: 0.2714 - val_loss: 2.8185 - val_sparse_categorical_accuracy: 0.2546

Epoch 00002: val_loss improved from 3.06808 to 2.81846, saving model to /model2.hdf5
Epoch 3/100
1168/1168 [==============================] - 329s 282ms/step - loss: 2.5589 - sparse_categorical_accuracy: 0.3253 - val_loss: 2.6020 - val_sparse_categorical_accuracy: 0.2913

Epoch 00003: val_loss improv

In [0]:
segments_len  = len(segments)
n_timesteps = segments[segments_len-1].shape[0]

# define LSTM configuration - hyperparameters

n_neurons = 150

n_batch_size = 25
n_epoch = 100

# create LSTM
model = Sequential()
model.add(Bidirectional(LSTM(n_neurons, input_shape=(n_timesteps, 400), return_sequences=True)))
model.add(BatchNormalization())
model.add(TimeDistributed(Dense(100, activation="relu")))
model.add(TimeDistributed(Dense(48, activation="softmax")))

adam = optimizers.adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
# train LSTM
es = EarlyStopping(patience=2)
checkpointer = ModelCheckpoint(filepath='/model3.hdf5', verbose=1, save_best_only=True)

model.fit(segments, segment_labels, epochs=n_epoch, batch_size=n_batch_size, callbacks=[es,checkpointer], validation_split=0.2)
print(model.summary())
model.reset_states()

Train on 1168 samples, validate on 292 samples
Epoch 1/100
1168/1168 [==============================] - 104s 89ms/step - loss: 3.0918 - sparse_categorical_accuracy: 0.2090 - val_loss: 2.6364 - val_sparse_categorical_accuracy: 0.2749

Epoch 00001: val_loss improved from inf to 2.63642, saving model to /model3.hdf5
Epoch 2/100
1168/1168 [==============================] - 102s 87ms/step - loss: 2.2829 - sparse_categorical_accuracy: 0.3580 - val_loss: 2.2894 - val_sparse_categorical_accuracy: 0.3463

Epoch 00002: val_loss improved from 2.63642 to 2.28942, saving model to /model3.hdf5
Epoch 3/100
1150/1168 [============================>.] - ETA: 1s - loss: 1.9379 - sparse_categorical_accuracy: 0.4357
Epoch 00003: val_loss improved from 2.28942 to 2.08698, saving model to /model3.hdf5
Epoch 4/100
1168/1168 [==============================] - 102s 87ms/step - loss: 1.6601 - sparse_categorical_accuracy: 0.5052 - val_loss: 1.9667 - val_sparse_categorical_accuracy: 0.4129

Epoch 00004: val_loss i



---



# Functional Keras LSTM - with Reshape (Batch * Segment)

----------------------------


In [0]:
def get_frame_get_label(labelframe_breakfast,data_feat_train,i,k,start_idx,end_idx):          
  stratified_index_df = pd.DataFrame(labelframe_breakfast[i]).reset_index()
  stratified_index_df = stratified_index_df[stratified_index_df[0]!=0]
  stratified_index_df = stratified_index_df[stratified_index_df['index'].isin(list(range(start_idx,end_idx)))]
  stratified_index = stratified_index_df.groupby(0, group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(stratified_index_df))))).sample(n=N,replace=True).reset_index(drop=True)
  stratified_index.sort_values('index',inplace=True)
  index_list = list(stratified_index['index'])

  frame = data_feat_train[i][index_list]
  frame = np.array(frame)
  #frame = frame.reshape(1,N*400)

  get_label_list = np.array(labelframe_breakfast[i])
  label= get_label_list[index_list]
  label= np.array(list(set(label)))

  return frame,label

def get_frame_get_label_replace(labelframe_breakfast,data_feat_train,i,k,start_idx,end_idx):
  stratified_index_df = pd.DataFrame(labelframe_breakfast[i]).reset_index()
  stratified_index_df = stratified_index_df[stratified_index_df[0]!=0]
  stratified_index_df = stratified_index_df[stratified_index_df['index'].isin(list(range(start_idx,end_idx)))]
  stratified_index = stratified_index_df.groupby(0, group_keys=False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(stratified_index_df))),replace=True)).sample(n=N,replace=True).reset_index(drop=True)
  stratified_index.sort_values('index',inplace=True)
  index_list = list(stratified_index['index'])
  
  frame = data_feat_train[i][index_list]
  frame = np.array(frame)
  #frame = frame.reshape(1,N*400)

  get_label_list = np.array(labelframe_breakfast[i])
  label= get_label_list[index_list]
  label= np.array(list(set(label)))

  
  return frame,label

In [0]:
N = 200
segments = []
segment_labels = []

def train_gen():
  for i, video in tqdm(enumerate(data_feat_train)):
    #print("start video {}".format(i))
    for k in range(len(data_labels_loc[i])-1):
      if len(data_labels_loc[i])==1:
        start_idx = data_labels_loc[i][0]
        end_idx = len(labelframe_breakfast[i])
        len_idx = end_idx- start_idx
        print(len_idx)

        if  len_idx > N:
          frame,label = get_frame_get_label(labelframe_breakfast,data_feat_train,i,k,start_idx,end_idx)
        elif len_idx < N : 
          frame,label = get_frame_get_label_replace(labelframe_breakfast,data_feat_train,i,k,start_idx,end_idx)


      elif len(data_labels_loc[i])>1:
        if k==0:
          start_idx = data_labels_loc[i][k]
        else:
          start_idx = data_labels_loc[i][k]+1
        end_idx = data_labels_loc[i][k+1]
        len_idx = end_idx- start_idx
        
        if  len_idx> N:
          frame,label = get_frame_get_label(labelframe_breakfast,data_feat_train,i,k,start_idx,end_idx)
        elif len_idx< N : 
          frame,label = get_frame_get_label_replace(labelframe_breakfast,data_feat_train,i,k,start_idx,end_idx)
      
      segments.append(frame)
      segment_labels.append(label)
  
  return segments, segment_labels

In [0]:
segments, segment_labels =  train_gen()
segments = np.array(segments)
print(segments.shape)
segment_labels = np.array(segment_labels)
print(segment_labels.shape)

1460it [01:03, 22.88it/s]


(6965, 200, 400)
(6965, 1)


In [0]:
n_timesteps = segments.shape[1]
features = segments.shape[2]
n_neurons = 200

n_batch_size = 25
n_epoch = 100

adam = optimizers.adam(lr=0.0001)

inp = Input(shape=(n_timesteps,features))
tcl1 = Conv1D(filters=250,
               kernel_size=3,
               strides=1,
               activation='relu')(inp)
tcl1 = BatchNormalization()(tcl1)
tcl1 = TimeDistributed(Dense(150, activation="relu")) (tcl1)
max_pool_bf = MaxPooling1D(pool_size=2)(tcl1)
avg_pool_bf = AveragePooling1D(pool_size=2)(tcl1)
merge = concatenate([max_pool_bf, avg_pool_bf])

lstm = LSTM(n_neurons, return_sequences=True)(merge)
lstm = BatchNormalization()(lstm)

globalavg_pool = GlobalAveragePooling1D()(lstm)
globalmax_pool = GlobalMaxPooling1D()(lstm)
merge2 = concatenate([globalavg_pool, globalmax_pool])
outp = Dense(300, activation="relu")(merge2)
outp = Dropout(0.2)(outp)
outp = Dense(150, activation="relu")(outp)
outp = Dropout(0.2)(outp)
outp = Dense(48, activation="softmax")(outp)
model = Model(inputs=inp, outputs=outp)

es = EarlyStopping(patience=4)
checkpointer = ModelCheckpoint(filepath='/model.hdf5', verbose=1, save_best_only=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
print(model.summary())
model.fit(segments, segment_labels, epochs=n_epoch, batch_size=n_batch_size, callbacks=[checkpointer,es], validation_split=0.2)
model.reset_states()

Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 200, 400)     0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 198, 250)     300250      input_16[0][0]                   
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 198, 250)     1000        conv1d_6[0][0]                   
__________________________________________________________________________________________________
time_distributed_10 (TimeDistri (None, 198, 150)     37650       batch_normalization_13[0][0]     
___________________________________________________________________________________________


---


# Archive


---



In [0]:
# define LSTM configuration
segments_len  = len(segments)
n_timesteps = segments[segments_len-1].shape[0]

# define LSTM configuration - hyperparameters

n_neurons = 400
n_neurons2 = 1000

n_batch_size = 25
n_epoch = 100

# create LSTM
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(n_timesteps, 400), return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(n_neurons2, return_sequences=True))

model.add(BatchNormalization())
model.add(TimeDistributed(Dense(400, activation="relu")))
model.add(TimeDistributed(Dense(48, activation="softmax")))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['sparse_categorical_accuracy'])
print(model.summary())
# train LSTM
es = EarlyStopping(patience=2)
checkpointer = ModelCheckpoint(filepath='/model.hdf5', verbose=1, save_best_only=True)

model.fit(segments, segment_labels, epochs=n_epoch, batch_size=n_batch_size, callbacks=[es,checkpointer], validation_split=0.2)
model.reset_states()







Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 800, 400)          1281600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 800, 400)          1600      
_________________________________________________________________
lstm_2 (LSTM)                (None, 800, 1000)         5604000   
_________________________________________________________________
batch_normalization_2 (Batch (None, 800, 1000)         4000      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 800, 400)          400400    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 800, 48)           19248     
Total params: 7,310,848
Trainable params: 7,308,048
Non-trainable params: 2,800
__________________________________

In [0]:
# define LSTM configuration
segments_len  = len(segments)
n_timesteps = segments[segments_len-1].shape[0]

# define LSTM configuration - hyperparameters

n_neurons = 400
n_neurons2 = 1000

n_batch_size = 25
n_epoch = 100

# create LSTM
model = Sequential()
model.add(Bidirectional(LSTM(n_neurons, input_shape=(n_timesteps, 400), return_sequences=True)))
model.add(BatchNormalization())
model.add(LSTM(n_neurons2, return_sequences=True))

model.add(BatchNormalization())
model.add(TimeDistributed(Dense(400, activation="relu")))
model.add(TimeDistributed(Dense(48, activation="softmax")))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['sparse_categorical_accuracy'])
# train LSTM
es = EarlyStopping(patience=2)
checkpointer = ModelCheckpoint(filepath='/model2.hdf5', verbose=1, save_best_only=True)

model.fit(segments, segment_labels, epochs=n_epoch, batch_size=n_batch_size, callbacks=[es,checkpointer], validation_split=0.2)
print(model.summary())
model.reset_states()



In [0]:
segments_len  = len(segments)
n_timesteps = segments[segments_len-1].shape[0]

# define LSTM configuration - hyperparameters

n_neurons = 400
n_neurons2 = 1000

n_batch_size = 25
n_epoch = 100

# create LSTM
model = Sequential()
model.add(Bidirectional(LSTM(n_neurons, input_shape=(n_timesteps, 400), return_sequences=True)))
model.add(BatchNormalization())
model.add(TimeDistributed(Dense(400, activation="relu")))
model.add(TimeDistributed(Dense(48, activation="softmax")))

adam = optimizers.adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
# train LSTM
es = EarlyStopping(patience=2)
checkpointer = ModelCheckpoint(filepath='/model2.hdf5', verbose=1, save_best_only=True)

model.fit(segments, segment_labels, epochs=n_epoch, batch_size=n_batch_size, callbacks=[es,checkpointer], validation_split=0.2)
print(model.summary())
model.reset_states()

Train on 1168 samples, validate on 292 samples
Epoch 1/100
1168/1168 [==============================] - 327s 280ms/step - loss: 2.7771 - sparse_categorical_accuracy: 0.2650 - val_loss: 2.5890 - val_sparse_categorical_accuracy: 0.2992

Epoch 00001: val_loss improved from inf to 2.58904, saving model to /model2.hdf5
Epoch 2/100
1168/1168 [==============================] - 320s 274ms/step - loss: 1.8730 - sparse_categorical_accuracy: 0.4464 - val_loss: 2.2177 - val_sparse_categorical_accuracy: 0.3656

Epoch 00002: val_loss improved from 2.58904 to 2.21771, saving model to /model2.hdf5
Epoch 3/100
1168/1168 [==============================] - 316s 271ms/step - loss: 1.5304 - sparse_categorical_accuracy: 0.5291 - val_loss: 1.9374 - val_sparse_categorical_accuracy: 0.4393

Epoch 00003: val_loss improved from 2.21771 to 1.93742, saving model to /model2.hdf5
Epoch 4/100
1168/1168 [==============================] - 311s 266ms/step - loss: 1.3269 - sparse_categorical_accuracy: 0.5788 - val_loss: 

In [0]:
segments_len  = len(segments)
n_timesteps = segments[segments_len-1].shape[0]

# define LSTM configuration - hyperparameters

n_neurons = 200
n_neurons2 = 100


n_batch_size = 25
n_epoch = 100

# create LSTM
model = Sequential()
model.add(Bidirectional(LSTM(n_neurons, input_shape=(n_timesteps, 400), return_sequences=True)))
model.add(BatchNormalization())
model.add(LSTM(n_neurons2, return_sequences=True))
model.add(TimeDistributed(Dense(80, activation="relu")))
model.add(TimeDistributed(Dense(48, activation="softmax")))

adam = optimizers.adam()
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
# train LSTM
es = EarlyStopping(patience=2)
checkpointer = ModelCheckpoint(filepath='/model3.hdf5', verbose=1, save_best_only=True)

model.fit(segments, segment_labels, epochs=n_epoch, batch_size=n_batch_size, callbacks=[es,checkpointer], validation_split=0.2)
print(model.summary())
model.reset_states()

Train on 1168 samples, validate on 292 samples
Epoch 1/100
1168/1168 [==============================] - 130s 112ms/step - loss: 2.9785 - sparse_categorical_accuracy: 0.2290 - val_loss: 2.6086 - val_sparse_categorical_accuracy: 0.2913

Epoch 00001: val_loss improved from inf to 2.60861, saving model to /model3.hdf5
Epoch 2/100
1168/1168 [==============================] - 127s 109ms/step - loss: 2.2163 - sparse_categorical_accuracy: 0.3773 - val_loss: 2.2671 - val_sparse_categorical_accuracy: 0.3477

Epoch 00002: val_loss improved from 2.60861 to 2.26714, saving model to /model3.hdf5
Epoch 3/100
1168/1168 [==============================] - 127s 109ms/step - loss: 1.8497 - sparse_categorical_accuracy: 0.4620 - val_loss: 2.2571 - val_sparse_categorical_accuracy: 0.3723

Epoch 00003: val_loss improved from 2.26714 to 2.25713, saving model to /model3.hdf5
Epoch 4/100
1168/1168 [==============================] - 127s 108ms/step - loss: 1.6222 - sparse_categorical_accuracy: 0.5118 - val_loss: 